# Chapter 3 - Looking Inside Transformer LLMs

## Loading the LLM

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## The Inputs and Outputs of a Trained Transformer LLM

In [2]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happended."

output = generator(prompt)

print(output[0]['generated_text'])

You are not running the flash-attention implementation, expect numerical differences.




Dear Sarah,

I hope this email finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in your garden. I understand that it must have been a devastating experience


In [3]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

## Choosing a single token from the probability distribution (sampling/decoding)

In [4]:
prompt = "The capital of France is"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")
model_output = model.model(input_ids)

lm_head_output = model.lm_head(model_output[0])

In [5]:
token_id = lm_head_output[0, -1].argmax(-1)
tokenizer.decode(token_id)

'Paris'

In [6]:
model_output[0].shape

torch.Size([1, 5, 3072])

In [7]:
lm_head_output.shape

torch.Size([1, 5, 32064])

## Speeding up generation by caching keys and values

In [8]:
prompt = "Write a long email apoligizing to Nhan for the tragic gardening mishap. Explain how it happended."

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")


In [9]:
%%timeit -n 1
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=True
)

6.63 s ± 1.95 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit -n 1
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=False
)

28.5 s ± 694 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
